# Resumo do código

### <u>Código que gera o ficheiro Long_DataFusion</u>
---
O objectivo é receber dados dos Ninjas e da Delta e devolver um conjunto de métricas para estudar o comportamento de vários produtos em diversas lojas. 

---
- Inputs

> __Dados dos ninjas__ em formato Wide
> - 1's e 0's consoante a presença e ausência do produto, Data e Loja

> __Dados da Delta__ em formato Wide
> - Stocks e trânsito, Sellout dia anterior

- Outputs

> __Ficheiro Long__ (artigos na mesma coluna)

> __Métricas novas:__
> - Roturas de Stock, Roturas de Linear, Roturas consecutivas por dia, Roturas consecutivas por fim de semana
> - Sinal
> - Ciclos e Adequação de Stock
> - Restock


In [47]:
import pandas as pd
import numpy as np
import json

def escrever_excel(dataFrame, nomeFicheiro):
    dataFrame.to_excel('D:\\B&N Dados\\Delta\\Piloto\\%s.xlsx' %nomeFicheiro, index=False)
    
def ler_json(ficheiro):
    with open('D:\\B&N Dados\\Delta\\%s.json' %ficheiro, 'r') as file:
        mapa = json.load(file)
    return mapa



# Ler e Mudar nomes

- # `dfNinjas`

In [48]:
#Info Ninja
dfNinjas1=pd.read_excel("D:\\B&N Dados\\Delta\\Piloto\\1Ninjas1fds.xlsx")
dfNinjas2=pd.read_excel("D:\\B&N Dados\\Delta\\Piloto\\1Ninjas2fds.xlsx")
dfNinjas3=pd.read_excel("D:\\B&N Dados\\Delta\\Piloto\\1Ninjas3fds.xlsx")
dfNinjas4=pd.read_excel("D:\\B&N Dados\\Delta\\Piloto\\1Ninjas4fds.xlsx")

dataframes=[dfNinjas1,dfNinjas2,dfNinjas3,dfNinjas4]
dfNinjas=pd.concat(dataframes, ignore_index=True)
dfNinjas = dfNinjas.rename(columns={' Data de Resposta': 'DATA', "Código da loja":"STORE"})

In [49]:
mapa_lojas = dfNinjas.set_index(dfNinjas.columns[1])[dfNinjas.columns[0]].to_dict()

> Definir variáveis importantes
> - Se houver um ficheiro com os produtos

In [50]:
# Ler ficheiro para dataframe
df_produtos = pd.read_csv('D:\\B&N Dados\\Delta\\Piloto\\produtos.txt', header=None)

# Passar para uma lista
produtos = df_produtos[0].tolist()
resto = dfNinjas.columns.difference(produtos)

> - Se se quiser usar o primeiro e o último produtos

# Formato Long

In [51]:
dfNinjasLong = dfNinjas.melt(id_vars=resto, value_vars=produtos, var_name='DESC_ARTIGO', value_name='NinjaInfo')

- Altura do dia

In [52]:
# Hora para datetime
dfNinjasLong['Hora']= pd.to_datetime(dfNinjasLong['Hora'], format='%H:%M:%S', errors='coerce').dt.time  

#coluna nova para data e hora
dfNinjasLong['DataComp'] = pd.to_datetime(dfNinjasLong['DATA'].dt.strftime('%Y-%m-%d') + ' ' + dfNinjasLong['Hora'].astype(str), errors='coerce') 

altura_do_dia = lambda x: 'Manhã' if x < 13 else ('Tarde' if x < 18 else ('Noite' if x < 24 else 'Indefinido'))

dfNinjasLong['Altura_do_Dia'] = dfNinjasLong['DataComp'].apply(lambda x: altura_do_dia(x.hour))
dfNinjasLong=dfNinjasLong.drop(columns=["DataComp"])

- # `dfDelta`

In [53]:
# Ler o ficheiro long com Stocks e Fornecimento
dfDelta=pd.read_csv("D:\\B&N Dados\\Delta\\Stocks\\StocksTotal\\Mil_Métricas_Piloto.csv")

# Renomear
dfDelta = dfDelta.rename(columns={"STORE_NAME":"Loja"})

In [54]:
dfDelta['DATA'] = pd.to_datetime(dfDelta['DATA'], format='%Y-%m-%d')

- Tirar tudo o que termina com "Dias_Antes"

In [55]:
dfDelta = dfDelta.filter(regex='^(?!.*_Dias_Antes$)')

> Reposição

In [56]:
# Ler o ficheiro com os nomes das lojas que fazem reposição
dfRepos=pd.read_excel("D:\\B&N Dados\\Delta\\Reposição_Sonae_Código.xlsx")

# Criar coluna que determina quais lojas têm reposição
dfDelta['Reposição'] = [1 if val in dfRepos['STORE'].values else 0 for val in dfDelta['STORE']]

# <font color="green">Estruturar a base de dados</font>

- Fazer o dataframe que servirá de modelo

In [57]:
import itertools

# Definir variáveis para cada coluna
dates = dfNinjasLong["DATA"].unique()
stores = dfNinjasLong["STORE"].unique()
desc_artigos = dfNinjasLong["DESC_ARTIGO"].unique()

# Criar todas as combinações possíveis
combinations = list(itertools.product(dates, stores, desc_artigos))

# Criar o DataDrame com as combinações possíveis
dfModelo = pd.DataFrame(combinations, columns=['DATA', 'STORE', 'DESC_ARTIGO'])

# Converter a coluna da "DATA" para datetime
dfModelo['DATA'] = pd.to_datetime(dfModelo['DATA'], format='%Y-%m-%d')


In [58]:
# Definir os valores que queremos para as alturas do dia
times = ['Manhã', 'Tarde', 'Noite']

# Criar uma lista de dfs, vai ter o número de alturas do dia certo
dfs = [dfModelo.assign(Altura_do_Dia=t) for t in times]

# Juntar tudo no mesmo dataframe
df_Mergir = pd.concat(dfs, ignore_index=True)


> `MERGIR`

In [59]:
dfNinjasLong = pd.merge(df_Mergir, dfNinjasLong, on=['DATA', 'STORE', 'DESC_ARTIGO', 'Altura_do_Dia'], how='left')

dfNinjasLong = dfNinjasLong.sort_values(by=["STORE","DESC_ARTIGO","DATA"])

- Semana

In [60]:
dfNinjasLong['Semana'] = dfNinjasLong['DATA'].dt.isocalendar().week

dfNinjasLong['Semana'] = dfNinjasLong.groupby('Semana').ngroup() + 1

- Dia

In [61]:
nome_dia = dfNinjasLong['DATA'].dt.day_name().map({'Friday': 'Sexta', 'Saturday': 'Sábado', 'Sunday': 'Domingo'})

dfNinjasLong['Dia'] = nome_dia

## <font color=green> Fim base </font>

---

In [62]:
dfNinjasLong['Nome da Loja'] = dfNinjasLong['Nome da Loja'].fillna(dfNinjasLong['STORE'].map(mapa_lojas))
dfNinjasLong.head()

,DATA,STORE,DESC_ARTIGO,Altura_do_Dia,Hora,Nome da Loja,NinjaInfo,Semana,Dia
0,2023-04-14,1,BEBIDA CEREAIS DELTA C/20%CAFE FR 200G,Manhã,10:39:00,Continente Matosinhos,1.0,1,Sexta
1200,2023-04-14,1,BEBIDA CEREAIS DELTA C/20%CAFE FR 200G,Tarde,14:46:00,Continente Matosinhos,1.0,1,Sexta
2400,2023-04-14,1,BEBIDA CEREAIS DELTA C/20%CAFE FR 200G,Noite,19:08:00,Continente Matosinhos,1.0,1,Sexta
100,2023-04-15,1,BEBIDA CEREAIS DELTA C/20%CAFE FR 200G,Manhã,10:56:25,Continente Matosinhos,1.0,1,Sábado
1300,2023-04-15,1,BEBIDA CEREAIS DELTA C/20%CAFE FR 200G,Tarde,15:19:35,Continente Matosinhos,1.0,1,Sábado


#  <span style="color:red"><u> Mergir</u> </span>

- # `dfFinal`

In [63]:
dfMeio = pd.merge(dfNinjasLong, dfDelta, how="left", on = ["DATA","STORE", "DESC_ARTIGO"])
dfMeio = dfMeio.drop(columns=["Loja"])


dfFinal=dfMeio.sort_values(by=["STORE","DESC_ARTIGO","DATA"]).copy()

>Sinal

In [65]:
# Categorias possíveis
mapping = {1: "Presente com Stock",
           2: "Ausente com Stock",
           3: "Presente sem Stock",
           4: "Ausente sem Stock",
           5: "Presente sem Registo",
           6: "Ausente sem Registo"}

# Definir coluna de sinal
dfFinal["Sinal"] = np.where((dfFinal["STOCK"] > 0) & (dfFinal["NinjaInfo"] == 1), 1,
                     np.where((dfFinal["STOCK"] > 0) & (dfFinal["NinjaInfo"] == 0), 2,
                     np.where((dfFinal["STOCK"] <= 0) & (dfFinal["NinjaInfo"] == 1), 3,
                     np.where((dfFinal["STOCK"] <= 0) & (dfFinal["NinjaInfo"] == 0), 4,
                     np.where((dfFinal["STOCK"].isna()) & (dfFinal["NinjaInfo"] == 1), 5,
                     np.where((dfFinal["STOCK"].isna()) & (dfFinal["NinjaInfo"] == 0), 6,
                     np.nan))))))

# Substituir números pelas expressões escolhidas
dfFinal["Sinal"] = dfFinal["Sinal"].map(mapping)


# <span style="color:Blue">Outras métricas</span>

> Reabastecimento

In [75]:
dfFinal["Restock"] = np.where(dfFinal["STOCK"] > dfFinal["STOCK"].shift(), "Houve reabastecimento",
                                np.where((dfFinal["STOCK"] <= 0) & (dfFinal["STOCK"].shift() > 0), "Houve rotura", 0))

#Fazer um groupby

dfFinal.Restock.value_counts()


0                        3278
Houve reabastecimento     320
Houve rotura                2
Name: Restock, dtype: int64

In [66]:
dfFinal["Restock"] = dfFinal.groupby("DESC_ARTIGO").apply(
    lambda group: np.where(
        group["STOCK"] > group["STOCK"].shift(), "Houve reabastecimento",
        np.where((group["STOCK"] <= 0) & (group["STOCK"].shift() > 0), "Houve rotura", 0)
    )
).reset_index(drop=True)

In [77]:
dfFinal[['DATA', 'STORE', 'DESC_ARTIGO', 'Altura_do_Dia', 'Hora', 'Nome da Loja', 'NinjaInfo', 'Semana', 'Dia', 'Restock']].head()

,DATA,STORE,DESC_ARTIGO,Altura_do_Dia,Hora,Nome da Loja,NinjaInfo,Semana,Dia,Restock
0,2023-04-14,1,BEBIDA CEREAIS DELTA C/20%CAFE FR 200G,Manhã,10:39:00,Continente Matosinhos,1.0,1,Sexta,0
1,2023-04-14,1,BEBIDA CEREAIS DELTA C/20%CAFE FR 200G,Tarde,14:46:00,Continente Matosinhos,1.0,1,Sexta,0
2,2023-04-14,1,BEBIDA CEREAIS DELTA C/20%CAFE FR 200G,Noite,19:08:00,Continente Matosinhos,1.0,1,Sexta,0
3,2023-04-15,1,BEBIDA CEREAIS DELTA C/20%CAFE FR 200G,Manhã,10:56:25,Continente Matosinhos,1.0,1,Sábado,0
4,2023-04-15,1,BEBIDA CEREAIS DELTA C/20%CAFE FR 200G,Tarde,15:19:35,Continente Matosinhos,1.0,1,Sábado,0


- Roturas Reais (Ninjas)

In [118]:
#1 Quando detectada rotura
dfFinal["Roturas_Linear_dia"] = dfFinal[dfFinal['NinjaInfo'] == 0].groupby(['STORE', 'DATA', 'DESC_ARTIGO'])['NinjaInfo'].transform('count')

#0 Quando não há rotura
dfFinal["Roturas_Linear_dia"] = np.where(dfFinal['NinjaInfo'] == 1, 0, dfFinal['Roturas_Linear_dia'])

- Roturas consecutivas por dia

In [129]:
#Detectar em quais é que há consecutivas
dfFinal['Roturas_Consecutivas'] = dfFinal.groupby(['STORE', 'Semana', 'DESC_ARTIGO'], group_keys=False)['NinjaInfo'].apply(lambda x: ((x == 0) & (x.shift(1) == 0)).astype(int))


#Há um problema na linha de baixo! só quero que some a soma
#Somar para saber o total
dfFinal['Roturas_Consecutivas'] = dfFinal.groupby(['STORE', 'Semana', 'DESC_ARTIGO'], group_keys=False)['Roturas_Consecutivas'].apply(
    lambda x: x.groupby((x != x.shift()).cumsum()).cumsum() if any(x == 1) else x)



In [131]:
dfFinal[['DATA', 'DESC_ARTIGO', 'Altura_do_Dia', 'Nome da Loja', 'NinjaInfo', 'Semana', 'Dia', 'Roturas_Consecutivas']][dfFinal["NinjaInfo"]==0].head(40)

,DATA,DESC_ARTIGO,Altura_do_Dia,Nome da Loja,NinjaInfo,Semana,Dia,Roturas_Consecutivas
424,2023-05-05,CAFÉ BELLISSIMO INTENSO 200GR,Tarde,Continente Amadora,0.0,4,Sexta,0
425,2023-05-05,CAFÉ BELLISSIMO INTENSO 200GR,Noite,Continente Amadora,0.0,4,Sexta,1
426,2023-05-06,CAFÉ BELLISSIMO INTENSO 200GR,Manhã,Continente Amadora,0.0,4,Sábado,2
427,2023-05-06,CAFÉ BELLISSIMO INTENSO 200GR,Tarde,Continente Amadora,0.0,4,Sábado,3
428,2023-05-06,CAFÉ BELLISSIMO INTENSO 200GR,Noite,Continente Amadora,0.0,4,Sábado,4
429,2023-05-07,CAFÉ BELLISSIMO INTENSO 200GR,Manhã,Continente Amadora,0.0,4,Domingo,5
750,2023-05-06,BEBIDA CEREAIS DELTA C/20%CAFE FR 200G,Manhã,Continente Cascais,0.0,4,Sábado,0
821,2023-05-05,CAFÉ DELTA Q MYTHIQ 80CAP,Noite,Continente Cascais,0.0,4,Sexta,0
1152,2023-04-14,CAFÉ DELTA Q MYTHIQ 80CAP,Manhã,Continente Leiria,0.0,1,Sexta,0
1153,2023-04-14,CAFÉ DELTA Q MYTHIQ 80CAP,Tarde,Continente Leiria,0.0,1,Sexta,1


- Roturas consecutivas entre dias

In [79]:
#Detectar Roturas consecutivas entre dias do mesmo fim de semana
dfFinal['Roturas_Consecutivas_fds'] = dfFinal.groupby(['STORE', dfFinal['DATA'].dt.isocalendar().week, 'DESC_ARTIGO'], group_keys=False)['NinjaInfo'].apply(lambda x: ((x == 0) & (x.shift(1) == 0)).astype(int))
dfFinal['Roturas_Consecutivas_fds'] = dfFinal.groupby(['STORE', dfFinal['DATA'].dt.isocalendar().week, 'DESC_ARTIGO'], group_keys=False)['Roturas_Consecutivas_fds'].transform(lambda x: (x.cumsum() if any(x == 1) else x))
#Eliminar valores repetidos quando não está a haver rotura
dfFinal['Roturas_Consecutivas_fds'] =np.where(dfFinal["Roturas_Linear_dia"] == 0, 0, dfFinal['Roturas_Consecutivas_fds'])

KeyError: 'Roturas_Linear_dia'

- Proporção

In [43]:
dfFinal['Proporção_Consecutivas_dia'] = dfFinal.groupby(['STORE', 'DATA', 'DESC_ARTIGO'], group_keys=False)['NinjaInfo'].transform('mean')
dfFinal['Std_Consecutivas_dia'] = dfFinal.groupby(['STORE', 'DATA', 'DESC_ARTIGO'], group_keys=False)['NinjaInfo'].transform('std')

dfFinal['Proporção_Consecutivas_fds'] = dfFinal.groupby(['Semana', 'STORE', 'DESC_ARTIGO'], group_keys=False)['NinjaInfo'].transform('mean')
dfFinal['Std_Consecutivas_fds'] = dfFinal.groupby(['Semana', 'STORE', 'DESC_ARTIGO'], group_keys=False)['NinjaInfo'].transform('std')

In [44]:
dfFinal.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3600 entries, 0 to 3599
Data columns (total 93 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   DATA                         3600 non-null   datetime64[ns]
 1   STORE                        3600 non-null   int64         
 2   DESC_ARTIGO                  3600 non-null   object        
 3   Altura_do_Dia                3600 non-null   object        
 4   Hora                         3010 non-null   object        
 5   Nome da Loja                 3600 non-null   object        
 6   NinjaInfo                    3010 non-null   float64       
 7   Semana                       3600 non-null   int64         
 8   Dia                          3600 non-null   object        
 9   EAN                          3600 non-null   int64         
 10  INTRANSIT                    3600 non-null   int64         
 11  EXPECTED                     3600 non-null 

In [45]:
dfFinal.info()
escrever_excel(dfFinal, "Paratestarcerto")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3600 entries, 0 to 3599
Data columns (total 93 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   DATA                         3600 non-null   datetime64[ns]
 1   STORE                        3600 non-null   int64         
 2   DESC_ARTIGO                  3600 non-null   object        
 3   Altura_do_Dia                3600 non-null   object        
 4   Hora                         3010 non-null   object        
 5   Nome da Loja                 3600 non-null   object        
 6   NinjaInfo                    3010 non-null   float64       
 7   Semana                       3600 non-null   int64         
 8   Dia                          3600 non-null   object        
 9   EAN                          3600 non-null   int64         
 10  INTRANSIT                    3600 non-null   int64         
 11  EXPECTED                     3600 non-null 

# Escrever

In [68]:
escrever_excel(dfFinal, "Long_DataFusion_PilotoCerto")